# Проект по классификации токсичных комментариев. BERT

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности.

Цель постройть модель со значением метрики качества *F1* не меньше 0.75.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
from transformers import BertTokenizer
from transformers import BertModel
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

In [41]:
pth1 = '/datasets/toxic_comments.csv'
pth2 = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

try:
    df = pd.read_csv(pth1)
except:
    df = pd.read_csv(pth2)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Избавимся от ненужного столбца, который копирует индексы

In [43]:
df = df.drop('Unnamed: 0', axis=1)

In [44]:
df['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

## Предобработка

### Токенизация

In [45]:
tokenizer = BertTokenizer.from_pretrained("unitary/toxic-bert")
model = BertModel.from_pretrained("unitary/toxic-bert")

In [46]:
tqdm.pandas()
tokenized = df['text'].apply(lambda x: tokenizer.encode(x, max_length=128, truncation=True, add_special_tokens=True))
padded = pad_sequence([torch.as_tensor(seq) for seq in tokenized], batch_first=True)

attention_mask = padded > 0
attention_mask = attention_mask.type(torch.LongTensor)

In [47]:
dataset = TensorDataset(attention_mask, padded)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(f'Device: {device}')

Device: cuda:0


### Создание эмбедингов

In [48]:
try:
  features = pd.read_parquet('/content/drive/MyDrive/features.parquet')
except:
  embeddings = []
  model.to(device)
  model.eval()
  for attention_mask, padded in tqdm(dataloader):
      attention_mask, padded = attention_mask.to(device), padded.to(device)

      with torch.no_grad():
          batch_embeddings = model(padded, attention_mask=attention_mask)

      embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  features = np.concatenate(embeddings)
# pd.DataFrame(features).to_parquet('features.parquet')

### Делим выборки

In [49]:
X_train, X_test, y_train, y_test = train_test_split(features, df['toxic'], test_size=0.2, random_state=42)

## Обучение

### LogisticRegression

In [50]:
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200)

In [54]:
y_pred = model.predict(X_test)

In [55]:
f1_logreg = f1_score(y_test, y_pred)
print("f1_score:", f1_logreg)

f1_score: 0.9376479873717443


In [58]:
report = classification_report(y_test, y_pred)
print('Отчет о классификации:\n', report)


conf_matrix = confusion_matrix(y_test, y_pred)
print('Матрица путаницы:\n', conf_matrix)

Отчет о классификации:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     28658
           1       0.95      0.93      0.94      3201

    accuracy                           0.99     31859
   macro avg       0.97      0.96      0.97     31859
weighted avg       0.99      0.99      0.99     31859

Матрица путаницы:
 [[28494   164]
 [  231  2970]]


### RandomForestClassifier

In [59]:
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [60]:
y_pred = model.predict(X_test)

In [62]:
f1_rfc = f1_score(y_test, y_pred)
print("f1_score:", f1_rfc)

f1_score: 0.9352791878172588


In [61]:
report = classification_report(y_test, y_pred)
print('Отчет о классификации:\n', report)


conf_matrix = confusion_matrix(y_test, y_pred)
print('Матрица путаницы:\n', conf_matrix)

Отчет о классификации:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     28658
           1       0.95      0.92      0.94      3201

    accuracy                           0.99     31859
   macro avg       0.97      0.96      0.96     31859
weighted avg       0.99      0.99      0.99     31859

Матрица путаницы:
 [[28503   155]
 [  253  2948]]


### LGBMClassifier

In [63]:
model = lgb.LGBMClassifier()
model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 12985, number of negative: 114448
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.597133 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 127433, number of used features: 768
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.101897 -> initscore=-2.176326
[LightGBM] [Info] Start training from score -2.176326


LGBMClassifier()

In [64]:
y_pred = model.predict(X_test)

In [65]:
f1_lgb = f1_score(y_test, y_pred)
print("f1_score:", f1_lgb)

f1_score: 0.9367128463476071


In [66]:
report = classification_report(y_test, y_pred)
print('Отчет о классификации:\n', report)


conf_matrix = confusion_matrix(y_test, y_pred)
print('Матрица путаницы:\n', conf_matrix)

Отчет о классификации:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     28658
           1       0.94      0.93      0.94      3201

    accuracy                           0.99     31859
   macro avg       0.97      0.96      0.96     31859
weighted avg       0.99      0.99      0.99     31859

Матрица путаницы:
 [[28482   176]
 [  226  2975]]


### Сравнение

In [67]:
pd.DataFrame({'f1_score': [f1_lgb, f1_rfc, f1_logreg]},
             index=['LGBMClassifier', 'RandomForest', 'LogisticRegression']).round(3)

,f1_score
LGBMClassifier,0.937
RandomForest,0.935
LogisticRegression,0.938


### DummyRegressor

In [71]:
dum_prediction = pd.Series(1, index=y_test)

print(f1_score(y_test, dum_prediction))

0.18260125499144325


Как мы видим, константная модель которая помечает все строки токсичными, имеет точность всего лишь 0.18

## Выводы
Задачей проекта было разработать прототип модели машинного обучения, который мог бы предсказывать токсичность комментариев. В ходе работы были выполнены следующие шаги:
- Подготовка данных. В процессе подготовки данных были выполнены операции, такие как чтение и изучение данных, удаление лишнего стоблца. Для получения эмбеддингов текстовых данных была использована предобученная модель BERT.
- Анализ:
    - В данных имеется явный дисбаланс, устранили его с помощью встроенных методов весов в моделях машинного обучения.
- Перед обучением были разделены выборки в соотношении 1 к 5.
- Были обучены и протестированны три модели на кроссвалидации LGBMClassifier, LogisticRegression и RandomForest
- Экспериментировал с различными гиперпараметрами модели, такими как размерность эмбеддингов, размер батча и количество эпох.  
- Самой удачной для нашей задачи является комбинация toxic-bert эмбендингов и модели LogisticRegression, что дало F1 = 0.94, это полностью удовлетворяет условиям задачи. Данный подход должен хорошо работать на больших наборах данных и легко интерпретироваться.  

